In [1]:
import pandas as pd
import re
import os
import codecs

In [4]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

# Example usage
#testcode = 'C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items\\Inferential statistics\\uva-inferential statistics-107-nl.Rmd\\uva-inferential statistics-107-nl.Rmd'
#testcode = 'C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items\\Inferential statistics\\uva-inferential statistics-104-nl.Rmd\\uva-inferential statistics-104-nl.Rmd'
#testcode = 'C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items\\Inferential statistics\\uva-inferential statistics-104-nl_corrected.Rmd\\uva-inferential statistics-104-nl_corrected.Rmd'
#testcode = 'C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items\\Descriptive statistics\\uva-descriptive statistics-124-nl.Rmd\\uva-descriptive statistics-124-nl.Rmd'
#testcode = 'C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items\\Inferential statistics\\uva-inferential statistics-269-nl.Rmd\\uva-inferential statistics-269-nl.Rmd'
#testcode = 'C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items\\Descriptive statistics\\uva-descriptive statistics-124-nl_corrected.Rmd\\uva-descriptive statistics-124-nl_corrected.Rmd'
#file_encoding = detect_encoding(testcode)
#print(file_encoding)

#print(f"The detected encoding for {testcode} is: {file_encoding}")

# Directories

In [9]:
#multiple choice dir's
mc_directory_pc = "C:\\Users\\stenp\\Documents\\Werk\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items"
mc_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items"
mc_ssd = "E:\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items"

# Openstring/probability
osp_directory_laptop = "C:\\Users\\nicol\\Documents\\Werk\\StatistiekFabriekItemConversion\\OpenString\\OpenString_Items\\Probability"

# Openstring/Inferential Statistics
osi_directory_laptop = "C:\\Users\\nicol\\Documents\\Werk\\StatistiekFabriekItemConversion\\OpenString\\OpenString_Items\\Inferential Statistics"

# Inladen en corrigeren conversietabel

In [10]:
pd.set_option('display.max_colwidth', 100)
taxonomy_conversion_table = pd.read_csv("taxonomyLookupTablecopy.csv", sep = ';')
#taxonomy_conversion_table['oldTax'] = taxonomy_conversion_table['oldTax'].str.replace('Inferenti?le', 'Inferentiële')
#taxonomy_conversion_table['oldTax'] = taxonomy_conversion_table['oldTax'].str.replace('inferenti?le', 'inferentiële')
print(taxonomy_conversion_table[taxonomy_conversion_table['oldTax']=='exsection: Inferentiële statistiek,Anova,Twoway + interactie '].index)

Index([40], dtype='int64')


# Converteren Multiple choice vragen

In [34]:
def tax_converter_MC(path, table):
    if path.endswith('_corrected.Rmd'):
        return None

    corrected_file = path.replace('.Rmd', '_corrected.Rmd')  # Define the corrected_file before using it
    corrected_dir = os.path.dirname(corrected_file)
    if not os.path.exists(corrected_dir):
        os.makedirs(corrected_dir)
       
    not_found = set()
    updated = False
    already_updated = False
    with open(path, 'r', encoding=detect_encoding(path)) as source_file, open(corrected_file, 'w', encoding= detect_encoding(path)) as corrected_file:
        for line in source_file:
            # Find the 'exsection:' line in the files
            if line.startswith('exsection:'):
                # Save the line and strip it from leading and trailing whitespace
                to_convert_line = line.strip()
                index_oldTax = table[table['oldTax'] == to_convert_line + ' '].index
                if not index_oldTax.empty:
                    new_line = table.loc[index_oldTax, 'newTax'].values[0]
                    corrected_file.write(new_line + '\n')
                    updated = True
                    continue
                elif index_oldTax.empty:
                    index_newTax = table[table['newTax'] == to_convert_line].index
                    if not index_newTax.empty:
                        corrected_file.write(line)
                        already_updated = True
                    else:
                        # Line is not found in the table at all, don't do anything with it and save its location
                        not_found.add(to_convert_line)
            else:
                corrected_file.write(line)
    if updated:
        return None
        #return print((path + ' updated correctly'))
    elif already_updated:
        return None
        #return print((path + ' already correctly updated'))
    else:
        print((path + ' not updated', not_found))
#Loop door files    
mc_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items"

for root, _, files in os.walk(mc_directory_laptop):
    for filename in files:
        if filename.endswith('.Rmd'):
            file_path = os.path.join(root, filename)  # Use os.path.join to construct the file path
            
            
            tax_converter_MC(file_path, taxonomy_conversion_table)
            
        else:
            continue


# Converteren Open String vragen

In [6]:
def tax_converter_OS(path, table):
    if path.endswith('_corrected.Rmd'):
        return None

    corrected_file = path.replace('.Rmd', '_corrected.Rmd')  # Define the corrected_file before using it
    corrected_dir = os.path.dirname(corrected_file)
    if not os.path.exists(corrected_dir):
        os.makedirs(corrected_dir)
       
    not_found = set()
    updated = False
    already_updated = False
    #encodingvar = detect_encoding(path)
    with open(path, 'r', encoding= detect_encoding(path)) as source_file, open(corrected_file, 'w', encoding= detect_encoding(path)) as corrected_file:
        for line in source_file:
            # Find the 'exsection:' line in the files
            if line.startswith('exsection:'):
                # Save the line and strip it from leading and trailing whitespace
                to_convert_line = line.strip()
                index_oldTax = table[table['oldTax'] == to_convert_line + ' '].index
                if not index_oldTax.empty:
                    new_line = table.loc[index_oldTax, 'newTax'].values[0]
                    corrected_file.write(new_line + '\n')
                    updated = True
                    continue
                elif index_oldTax.empty:
                    index_newTax = table[table['newTax'] == to_convert_line].index
                    if not index_newTax.empty:
                        corrected_file.write(line)
                        already_updated = True
                    else:
                        # Line is not found in the table at all, don't do anything with it and save its location
                        not_found.add(to_convert_line)
            else:
                corrected_file.write(line)
    if updated:
        #return None
        return print((path + ' updated correctly'))
    elif already_updated:
        #return None
        return print((path + ' already correctly updated'))
    else:
        print((path + ' not updated', not_found))

#Loop door files    
OS_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\OpenString\\OpenString_Items"

for root, _, files in os.walk(OS_directory_laptop):
    for filename in files:
        if filename.endswith('.Rmd'):
            file_path = os.path.join(root, filename)  # Use os.path.join to construct the file path
            
            
            tax_converter_OS(file_path, taxonomy_conversion_table)
            
        else:
            continue

C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\OpenString\OpenString_Items\Inferential Statistics\uva-anova-f-test-807\uva-anova-f-test-807-nl.Rmd already correctly updated
C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\OpenString\OpenString_Items\Inferential Statistics\uva-oneway-anova-806\uva-oneway-anova-806-nl.Rmd already correctly updated
C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\OpenString\OpenString_Items\Probability\uva-addition-rule-654\uva-addition-rule-654-nl.Rmd already correctly updated
C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\OpenString\OpenString_Items\Probability\uva-conditional-probability-687\uva-conditional-probability-687-nl.Rmd already correctly updated
C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\OpenString\OpenString_Items\Probability\uva-conditional-probability-917\uva-conditional-probability-917-nl.Rmd already correctly updated
C:\Users\stenp\Documents\Werk_C\Stati

# Vragen vinden met afbeelding

In [6]:
def image_qeustion(path):
    if path.endswith('_corrected.Rmd'):
        return None
    with open(path, 'r', encoding= detect_encoding(path)) as x:
        for line in x:
            if '.png' in line:
                return print(path)
            else:
                continue

mc_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items"
OS_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\OpenString\\OpenString_Items"
for root, _, files in os.walk(mc_directory_laptop):
    for filename in files:
        if filename.endswith('.Rmd'):
            file_path = os.path.join(root, filename)  # Use os.path.join to construct the file path
            
            
            image_qeustion(file_path)
            

C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\MultipleChoice\MultipleChoice_Items\Descriptive statistics\uva-descriptive statistics-124-nl.Rmd\uva-descriptive statistics-124-nl.Rmd
C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\MultipleChoice\MultipleChoice_Items\Descriptive statistics\uva-descriptive statistics-287-nl.Rmd\uva-descriptive statistics-287-nl.Rmd
C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\MultipleChoice\MultipleChoice_Items\Descriptive statistics\uva-descriptive statistics-288-nl.Rmd\uva-descriptive statistics-288-nl.Rmd
C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\MultipleChoice\MultipleChoice_Items\Descriptive statistics\uva-descriptive statistics-293-nl.Rmd\uva-descriptive statistics-293-nl.Rmd
C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\MultipleChoice\MultipleChoice_Items\Descriptive statistics\uva-descriptive statistics-294-nl.Rmd\uva-descriptive statistics-294-nl.Rmd
C:\Us

# Vragen met vraagteken midden in woord

In [ ]:
def image_qeustion(path):
    if path.endswith('_corrected.Rmd'):
        return None
    with open(path, 'r', encoding= detect_encoding(path)) as x:
        for line in x:
            if '.png' in line:
                return print(path)
            else:
                continue

mc_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items"
OS_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\OpenString\\OpenString_Items"
for root, _, files in os.walk(mc_directory_laptop):
    for filename in files:
        if filename.endswith('.Rmd'):
            file_path = os.path.join(root, filename)  # Use os.path.join to construct the file path
            
            
            image_qeustion(file_path)

# vragen zonder exsection uberhaupt

In [8]:
def find_files_without_exsection(path):
    if path.endswith('_corrected.Rmd'):
        return None
    
    with open(path, 'r', encoding=detect_encoding(path)) as file:
        has_exsection = any('exsection:' in line for line in file)
    
    if not has_exsection:
        print(path)

mc_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items"
OS_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\OpenString\\OpenString_Items"
for root, _, files in os.walk(OS_directory_laptop):
    for filename in files:
        if filename.endswith('.Rmd'):
            file_path = os.path.join(root, filename)  # Use os.path.join to construct the file path
            
            
            find_files_without_exsection(file_path)

# vragen met ENTER MANUALLY

In [129]:
def image_qeustion(path):
    if path.endswith('_corrected.Rmd'):
        return None
    with open(path, 'r', encoding= detect_encoding(path)) as x:
        for line in x:
            if 'ENTER MANUALLY' in line:
                return print(path)
            else:
                continue

mc_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items"
OS_directory_laptop = "C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\OpenString\\OpenString_Items"
for root, _, files in os.walk(mc_directory_laptop):
    for filename in files:
        if filename.endswith('.Rmd'):
            file_path = os.path.join(root, filename)  # Use os.path.join to construct the file path
            
            
            image_qeustion(file_path)

C:\Users\stenp\Documents\Werk_C\StatistiekFabriekItemConversion\MultipleChoice\MultipleChoice_Items\Probability\uva-probability-150-nl.Rmd\uva-probability-150-nl.Rmd


# Notitites
je vindt ze niet doordat er in de oldTax een ' ' (spatie volgt) <br>

## Niet gevonden in conversietabel 
### Te verbeteren in csv file Multiple Choice
<b> Al verbeterd </b><br>
'exsection: Kansrekening,Kansrekening (ond.),Productregel' <b>verbeterd</b><br>
'exsection: Kansrekening,Kansrekening (ond.),Somregel' <b>verbeterd</b><br>
'exsection: Kansrekening,Kansdefinities,Kansverdeling,Normale verdeling,Onafhankelijkheid van twee of meer gebeurtenissen' <b>verbeterd</b><br>
'exsection: Kansrekening,Kansverdeling,Voorwaardelijke kansen' <b>verbeterd</b><br>
'exsection: Distributions/Discrete/Binomial'<b>verbeterd</b><br>


<b>Suggesties</b><br>
'exsection: Kansrekening,Kansverdeling,Uniforme verdeling' --> Distributions/Continuous/Uniform<br> 
'exsection: Kansrekening,Kansverdeling,Binomiale verdeling'--> Distributions/Discrete/Binomial <br> 

<b> Nog handmatig te doen </b><br>
'exsection: Kansrekening,Kansdefinities,Gebeurtenis'<br>
'exsection: Kansrekening,Kansrekening (ond.),Kansverdeling,Kansverdeling steekproefgemiddelde (onafhankelijke,Continue stochastische variabelen'<br>
'exsection: Kansrekening,Kansdefinities,Kansrekening (ond.),Somregel,Onafhankelijkheid van twee of meer gebeurtenissen'<br>
None<br>
'exsection: Kansrekening,Kansrekening (ond.),Optelregel voor kans op vereniging van twee gebeur'<br>
'exsection: Kansrekening,Kansrekening (ond.),Kansverdeling,Normale verdeling,Productregel'<br> 
'exsection: Kansrekening,Kansdefinities,Klassieke kansdefinitie,Gebeurtenis'<br>
'exsection: Kansrekening,Kansrekening (ond.),Kansverdeling,Kansverdeling steekproefgemiddelde (onafhankelijke,Discrete stochastische variabelen (elementair begr'<br> 
'exsection: Kansrekening,Kansdefinities,Kansverdeling,Voorwaardelijke kansen,Onafhankelijkheid van twee of meer gebeurtenissen'<br> 
'exsection: Inferentiële statistiek,Toetstheorie,Hypotheses (nul en alternatief; één- en tweezijdig'}

### Te verbeteren in csv file Open String Probability 
<b>Allemaal al correct, alleen achterhalen wat None betekent </b><br>
'exsection: Probability/Elementary Probability/Random variables/Expected value'<br>
'exsection: Probability/Elementary Probability/Conditional probability'<br>
'None'<br>
'exsection: Probability/Elementary Probability/General Rules/Union'<br>
'exsection: Probability/Elementary Probability/Random variables/Rules for expected values'<br>
'exsection: Probability/Elementary Probability/General Rules/Addition rule'<br>
'exsection: Probability/Elementary Probability/General Rules/Multiplication rule'<br>

### Te verbeteren in csv file Open String Inferential Statistics
'None'

# test met omzetten in veilige map


In [3]:
def remove_suffix_from_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith("_corrected.Rmd"):
                original_path = os.path.join(root, file)
                new_name = file.replace("_corrected.Rmd", ".Rmd")
                new_path = os.path.join(root, new_name)
                os.rename(original_path, new_path)
                print(f'Renamed: {original_path} to {new_path}')

#directory_path = "C:\\Users\\stenp\\Documents\\Werk_2\\MultipleChoice_Items\\Descriptive statistics\\corrected_files"
#directory_path = "C:\\Users\\stenp\\Documents\\Werk_2\\MultipleChoice_Items\\Inferential statistics\\corrected_files"
#directory_path = "C:\\Users\\stenp\\Documents\\Werk_2\\MultipleChoice_Items\\Probability\\corrected_files"
#directory_path = "C:\\Users\\stenp\\Documents\\Werk_2\\OpenString_Items"
remove_suffix_from_files(directory_path)

Renamed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-addition-rule-654\uva-addition-rule-654-nl_corrected.Rmd to C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-addition-rule-654\uva-addition-rule-654-nl.Rmd
Renamed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-conditional-probability-687\uva-conditional-probability-687-nl_corrected.Rmd to C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-conditional-probability-687\uva-conditional-probability-687-nl.Rmd
Renamed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-conditional-probability-917\uva-conditional-probability-917-nl_corrected.Rmd to C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-conditional-probability-917\uva-conditional-probability-917-nl.Rmd
Renamed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-conditional-probability-918\uva-conditional-probability-918-nl_corrected.Rmd to C:\Users\stenp\Documents\Wer

# niet runnen destructief!!!

In [2]:
########def remove_and_rename_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if not file.endswith("_corrected.Rmd") and file.endswith(".Rmd"):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f'Removed: {file_path}')
directory_path = "C:\\Users\\stenp\\Documents\\Werk_2\\OpenString_Items\\Probability"
remove_and_rename_files(directory_path)

Removed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-addition-rule-654\uva-addition-rule-654-nl.Rmd
Removed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-conditional-probability-687\uva-conditional-probability-687-nl.Rmd
Removed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-conditional-probability-917\uva-conditional-probability-917-nl.Rmd
Removed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-conditional-probability-918\uva-conditional-probability-918-nl.Rmd
Removed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-elementary-probability-1007\uva-elementary-probability-1007-nl.Rmd
Removed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-elementary-probability-1008\uva-elementary-probability-1008-nl.Rmd
Removed: C:\Users\stenp\Documents\Werk_2\OpenString_Items\Probability\uva-elementary-probability-1010\uva-elementary-probability-1010-nl.Rmd
Removed: C:\Users\stenp\Documents

# Rotzooi code, niet runnen

In [ ]:
def tax_converter(path, table):
    if path.endswith('.Rmd'):
        # Process individual Rmd files
        corrected_file = path.replace('.Rmd', '_corrected.Rmd')

        # Skip processing if it's already a corrected file
        if corrected_file.endswith('_corrected.Rmd'):
            return

        not_found = set()
        updated = False
        with open(path, 'r') as source_file, open(corrected_file, 'w') as corrected_file:
            for line in source_file:
                # Find the 'exsection:' line in the files
                if line.startswith('exsection:'):
                    # Save the line and strip it from leading and trailing whitespace
                    to_convert_line = line.strip()
                    index_oldTax = taxonomy_conversion_table[taxonomy_conversion_table['oldTax'] == to_convert_line + ' '].index
                    if not index_oldTax.empty:
                        new_line = taxonomy_conversion_table.loc[index_oldTax, 'newTax']
                        corrected_file.write(new_line + '\n')
                        updated = True
                        continue
                    elif index_oldTax.empty:
                        index_newTax = taxonomy_conversion_table[taxonomy_conversion_table['newTax'] == to_convert_line].index
                        if not index_newTax.empty:
                            corrected_file.write(line)
                            updated = True
                        else:
                            # Line is not found in the table at all, don't do anything with it and save its location
                            not_found.add(to_convert_line)
                else:
                    corrected_file.write(line)
        if not updated:
            return (path + ' not updated', not_found)
        else:
            return (path + ' correctly updated')

In [ ]:
#zoek door alle files in de variabele multiple choice map
for root, _, files in os.walk(mc_directory_laptop):
    for filename in files:
        if filename.endswith('.Rmd'):
            file_path = os.path.join(root, filename)  # Use os.path.join to construct the file path
            print(file_path)
            
            tax_converter_GPT(file_path, taxonomy_conversion_table)
            
        else:
            continue

In [ ]:
#tax_converter_GPT("E:\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items\\Inferential statistics\\uva-inferential statistics-250-nl.Rmd",taxonomy_conversion_table)
#tax_converter_GPT("C:\\Users\\stenp\\Documents\\Werk_C\\StatistiekFabriekItemConversion\\MultipleChoice\\MultipleChoice_Items\\Inferential statistics\\uva-inferential statistics-269-nl.Rmd\\uva-inferential statistics-269-nl.Rmd", taxonomy_conversion_table)